In [5]:
!pip install matplotlib scipy h5py


In [ ]:
import h5py
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
from functools import partial

t2m_kinematic_chain = [
    [0, 2, 5, 8, 11],
    [0, 1, 4, 7, 10],
    [0, 3, 6, 9, 12, 15],
    [9, 14, 17, 19, 21],
    [9, 13, 16, 18, 20],
]
HML_JOINT_NAMES = [
    "pelvis",
    "left_hip",
    "right_hip",
    "spine1",
    "left_knee",
    "right_knee",
    "spine2",
    "left_ankle",
    "right_ankle",
    "spine3",
    "left_foot",
    "right_foot",
    "neck",
    "left_collar",
    "right_collar",
    "head",
    "left_shoulder",
    "right_shoulder",
    "left_elbow",
    "right_elbow",
    "left_wrist",
    "right_wrist",
]




In [ ]:
# load from hdf5 file

path = "/workspaces/time_qa-dataset/gen_data/data.hdf5"

motion_data = {}

with h5py.File(path, "r") as hdf5_file:
    # Access the 'motion' group
    group = hdf5_file["motion"]

    # Iterate over datasets in the group
    for key, item in group.items():
        if isinstance(item, h5py.Dataset):  # Check if the item is a dataset
            # Load the dataset back into the dictionary
            motion_data[key] = np.array(item)

    # Iterate over attributes of the group
    for key, value in group.attrs.items():
        # Load the attribute back into the dictionary
        motion_data[key] = value


# load motion file
motion = motion_data["motion"][0].transpose(2, 0, 1)  # trajectory 300 22 3


In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def custom_plot_3d_motion(path: str, kinematic_tree: np.ndarray, data: np.ndarray,normalize: bool = True, painting_features=[], labels: bool = False,label_vector: bool=False, display_vector: bool = False):
    fps = 30

    if normalize: 
        # Normalize data relative to the initial position of the root joint
        height_offset = data[:, :, 1].min()
        data[:, :, 1] -= height_offset
        # This code calculates the minimum value of the y-coordinate across all 
        # data points and subtracts it from all y-coordinates.
        # This effectively shifts the animation so that the lowest point (in terms of y-coordinate) is at y = 0.
        # This is useful if your data includes negative y-values or if you want the motion to appear as if it's starting from ground level.

        data[..., 0] -= data[:, 0:1, 0]  # Normalize x relative to the first joint
        data[..., 2] -= data[:, 0:1, 2]  # Normalize z relative to the first joint
        # used to normalize the x and z
        # coordinates relative to the initial position of the root joint in each frame.
        # This ensures that the motion is visualized relative to the starting position of this joint,
        # effectively centering the animation around the root joint’s motion in the horizontal plane.
    

    # Determine data ranges for better axes limits
    max_range = np.array([data[:,:,i].max() - data[:,:,i].min() for i in range(3)]).max() / 2.0
    mid_x = (data[:,:,0].max() + data[:,:,0].min()) * 0.5
    mid_z = (data[:,:,2].max() + data[:,:,2].min()) * 0.5

    fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(5, 5))
    
    def init_plot():
        # Dynamically set axes limits
        ax.set_xlim3d([mid_x - max_range, mid_x + max_range])
        ax.set_ylim3d([0, max_range * 2])  # Adjust this to ensure the floor is visible
        ax.set_zlim3d([mid_z - max_range, mid_z + max_range])
        ax.grid(False)
        ax.axis('off')

        # Floor plane
        floor_size = max_range * 2  # Ensures the floor is a square and adequately large
        floor_verts = np.array([
            [mid_x - floor_size, 0, mid_z - floor_size],
            [mid_x - floor_size, 0, mid_z + floor_size],
            [mid_x + floor_size, 0, mid_z + floor_size],
            [mid_x + floor_size, 0, mid_z - floor_size]
        ])
        floor = Poly3DCollection([floor_verts], alpha=0.3, facecolor='grey')
        ax.add_collection3d(floor)

    def update(frame):
        ax.clear()
        init_plot()
        ax.view_init(elev=100, azim=-90)  # Adjust view angles if necessary

        # Dynamic plot adjustments based on frame data
        for chain in kinematic_tree:
            ax.plot3D(data[frame, chain, 0], data[frame, chain, 1], data[frame, chain, 2], 
                      linewidth=2.0, color='orange')

        # Plot painting features with trajectories and text labels
        for feature in painting_features:
            if feature in HML_JOINT_NAMES:
                feat_index = HML_JOINT_NAMES.index(feature)
                # Plot trajectory
                ax.plot3D(data[:frame + 1, feat_index, 0], 
                          data[:frame + 1, feat_index, 1], 
                          data[:frame + 1, feat_index, 2], 
                          linewidth=2.0, color='blue')
                # Add text label with coordinates
                coord_text = None
                if labels:
                    coord_text = feature
                
                if label_vector:
                    coord_text = f"{feature} ({data[frame, feat_index, 0]:.2f}, {data[frame, feat_index, 1]:.2f}, {data[frame, feat_index, 2]:.2f})"
                
                if display_vector:
                    # Display vector from the root joint to the feature
                    ax.quiver(data[frame, 0, 0], data[frame, 0, 1], data[frame, 0, 2],
                              data[frame, feat_index, 0] - data[frame, 0, 0], 
                              data[frame, feat_index, 1] - data[frame, 0, 1], 
                              data[frame, feat_index, 2] - data[frame, 0, 2], 
                              color='green', arrow_length_ratio=0.1, linestyle='dotted')
                # coord_text = f"{feature} ({data[frame, feat_index, 0]:.2f}, {data[frame, feat_index, 1]:.2f}, {data[frame, feat_index, 2]:.2f})"
                if coord_text is not None:
                    ax.text(data[frame, feat_index, 0], data[frame, feat_index, 1], data[frame, feat_index, 2] + 0.1,  # slight offset for readability
                        coord_text, color='red', fontsize=10)

    ani = FuncAnimation(fig, update, frames=len(data), interval=1000/fps, repeat=False)
    ani.save(path, writer='ffmpeg', fps=fps)
    plt.close()



for i in [True, False]:
    custom_plot_3d_motion(f"test{i}.mp4", t2m_kinematic_chain, motion, normalize=i, painting_features=["left_foot"], display_vector=True)

In [32]:
from datasets import VerificationMode,  load_dataset

def _load_dataset_split(splits: list[str]):
    """Workaround to overcome the missing hf implementation of only dowloading the split shards"""
    files = {}
    KEY = "dasyd/time-qa"
    splits = ["val"]
    for split in splits:
        files[split] = f"{split}-*"
    return load_dataset(
        KEY,
        "binary",
        data_dir="binary",
        data_files=files,
        verification_mode=VerificationMode.NO_CHECKS,
        num_proc=len(splits),
    )


ds = _load_dataset_split(["val"]).with_format("numpy")

idx = 0
sample = ds["val"][idx]["trajectory"].transpose(2, 0, 1)  # trajectory 300 22 3

custom_plot_3d_motion(f"sample{idx}.mp4", t2m_kinematic_chain, sample, normalize=True, painting_features=["left_foot"], display_vector=False)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
